In [1]:
# app records all participants (also have CRFs)
# samples recieved recorded in tissue tracking log
# dna extraction recorded in carldata_kenny.xls (from nanodrop data) and in lab books
# genotyping data in genotyping.xls
# have made study_data_to_check.csv to check try and check what's going on
# would be nice to reconcile genotyping.xls better - not handling gremlins well. should not spend too much time
# though given how much else need to do!

In [2]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
%matplotlib inline
from scipy.stats import scoreatpercentile
import math

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
import psycopg2 as pg

def dict_from_db():
    """
    Returns a dict of dataframes from the database tables
    which are populated by the study application
    """
    conn = pg.connect("dbname='carlplaying' user='drcjar' password='drcjar_is_not_fake'")

    cursor = conn.cursor()
    cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
    e = cursor.fetchall()

    table_name_list = [i[0] for i in e]

    select_template = 'SELECT * FROM {table_name}'
    frames_dict = {}
    for tname in table_name_list:
        query = select_template.format(table_name = tname)
        frames_dict[tname] = pd.read_sql(query, conn)
    return frames_dict

ddf = dict_from_db() # dict of dataframes
def removed():
    """
    Print information about removed participants
    """
    rm = ddf['ipfjes_removalreason'][ddf['ipfjes_removalreason'].reason_fk_id.notnull()].copy() 

    # add participant_id
    ct_lookup = ddf['ipfjes_demographics'].loc[:,['patient_id', 'hospital_number']].copy()
    ct_lookup.index = ct_lookup['patient_id'].copy()
    ct_lookup = ct_lookup['hospital_number'].to_dict()
    rm['participant_id'] = rm['episode_id'].map(ct_lookup)

    # add participant type
    pt_lookup = ddf['ipfjes_studyparticipantdetails'].loc[:,['episode_id', 'participant_type']].copy()
    pt_lookup.index = pt_lookup['episode_id']
    pt_lookup = pt_lookup['participant_type'].to_dict()
    rm['case'] = rm['episode_id'].map(pt_lookup)
    
    removal_reasons = ddf['ipfjes_reasonforremoval']
    removal_reasons.index = removal_reasons['id']
    removal_reasons = removal_reasons['name'].to_dict()
    removal_reasons[4] = 'no response after 3 attempts' # to fix a coding duplicate
    rm['reason'] = rm['reason_fk_id'].map(removal_reasons)
    return rm

df = pd.read_csv('flat_dataframe.csv')
gt = pd.read_csv('genotyping.csv',header=None)

In [4]:
gt.columns = ['participant_id', 'genotype', 'location']

In [5]:
len(gt)

945

In [6]:
# run initially and then genotyping.csv 'fixed'
# gt[gt.participant_id.duplicated(keep=False)].sort_values(by='participant_id').to_csv('genotype_duplicates.csv')

In [7]:
gt[gt.participant_id.duplicated(keep=False)].sort_values(by='participant_id')

,participant_id,genotype,location
842,010056-1,Homozygous Allele 1,NaN
7,010056-1,Homozygous Allele 1,Box Eight
0,010082-1,Heterozygous,
682,010082-1,Heterozygous,Poor Quality Box Three
8,030062-1,Homozygous Allele 1,Box Eight
843,030062-1,Homozygous Allele 1,NaN
299,040028-,Heterozygous,Box seven
845,040028-,Heterozygous,NaN
812,050003-1,Homozygous Allele 1,Repeat Box
644,050003-1,Homozygous Allele 1,Poor Quality Box One


In [8]:
gt = gt.drop_duplicates(subset='participant_id')

In [9]:
gt.participant_id.nunique()

921

In [10]:
gt.participant_id.map(len).value_counts()

8    529
7    375
9      9
6      4
5      4
Name: participant_id, dtype: int64

In [11]:
gt.participant_id = gt.participant_id.astype(str)
gt.participant_id = gt.participant_id.str.replace('-1','')
gt.participant_id = gt.participant_id.str.replace('-2','')
gt.participant_id = gt.participant_id.str.replace('-3','')


gt.participant_id = gt.participant_id.str.replace('DR|SF|WP|RN|PB|GW|-| ','')
gt.loc[gt['participant_id'].map(len) == 7, 'participant_id'] = gt.loc[gt['participant_id'].map(len) == 7, 'participant_id'].str.replace('000','00')

In [19]:
gt.loc[gt['participant_id'] == '08003', 'participant_id'] = '080003'
gt.loc[gt['participant_id'] == '08005', 'participant_id'] = '080005'
gt.loc[gt['participant_id'] == '12004', 'participant_id'] = '120004'
gt.loc[gt['participant_id'] == '12006', 'participant_id'] = '120006'
gt.loc[gt['participant_id'] == '19006', 'participant_id'] = '190006'
gt.loc[gt['participant_id'] == '16018', 'participant_id'] = '160018'
gt.loc[gt['participant_id'] == '19007', 'participant_id'] = '190007'
gt.loc[gt['participant_id'] == '10004B', 'participant_id'] = '100004'


In [13]:
gt.loc[gt.participant_id.map(len) == 5, 'participant_id'] = '0' + gt.loc[gt.participant_id.map(len) == 5, 'participant_id']

# look at and drop duplicates again

In [14]:
gt.participant_id = gt.participant_id.str.replace('040150','040015')

In [15]:
gt = gt.drop_duplicates(subset='participant_id')

In [16]:
# hack to replace missing 0
df.participant_id = df.participant_id.astype(str)
df.loc[df.participant_id.map(lambda x: len(x)) == 5, 'participant_id'] = '0' + df.loc[df.participant_id.map(lambda x: len(x)) == 5, 'participant_id']

In [17]:
len(set(gt.participant_id) - set(df.participant_id) - set(removed().participant_id))


8

In [25]:
removed().participant_id.str[:2].value_counts()

10    11
06     9
01     8
20     7
03     6
08     5
04     5
09     4
05     3
19     2
15     2
13     2
02     1
14     1
07     1
Name: participant_id, dtype: int64

In [20]:
# it is unexpected to have participant ids in our genotyping that are not, apparently, in our study

set(gt.participant_id) - set(df.participant_id) - set(removed().participant_id)
# these are mostly folk whoe were never interviewed
# 160000 is unexplained though

{'100104', '130029', '130030', '130038', '160000', '190019', '210018'}

In [27]:
len(set(df.participant_id) - set(removed().participant_id) - set(gt.participant_id)) 
# these are people for whom either no sample was recieved, a sample was recieved and processed but dna too poor
# quality to genotype, or a sample was recieved and processed but has not been genotyped yet (n=90)

140

In [33]:
gt.to_csv('genotyping_cleaned.csv')

In [34]:
gt

,participant_id,genotype,location
0,010082,Heterozygous,
1,040067,Homozygous Allele 1,Box Eight
2,200024,Heterozygous,Box Eight
3,200023,Heterozygous,Box Eight
4,190009,Heterozygous,Box Eight
5,060033,Heterozygous,Box Eight
6,160034,Heterozygous,Box Eight
7,010056,Homozygous Allele 1,Box Eight
8,030062,Homozygous Allele 1,Box Eight
9,010055,Heterozygous,Box Eight
